Розглянемо розв'язок наступного рівняння:

$$y' = y(x + y) , x \in [0,1]$$
$$y(1)=1$$


#Розв'язок за допомогою PINN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN,self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1,64),
            nn.Tanh(),
            nn.Linear(64,64),
            nn.Tanh(),
            nn.Linear(64,1)
        )
        self.double()
    def forward(self,x):
        return self.net(x)

In [ ]:
x = (0 - 2)*torch.rand(10, 1, requires_grad = True) + 2
x

tensor([[1.5102],
        [1.5263],
        [1.7918],
        [0.7030],
        [1.6664],
        [0.4544],
        [0.9548],
        [1.2066],
        [1.0154],
        [0.9449]], grad_fn=<AddBackward0>)

In [ ]:
x = torch.rand(10, 1, requires_grad = True)
x

tensor([[0.5565],
        [0.8074],
        [0.4151],
        [0.8705],
        [0.0013],
        [0.5215],
        [0.3622],
        [0.4601],
        [0.2716],
        [0.6920]], requires_grad=True)

In [ ]:
def intial_condition():
    return torch.tensor([0])

def boundary_condition():
    return torch.tensor([1])

In [ ]:
def generate_training_data(num_points):
    x = torch.rand(num_points, 1, requires_grad = True)

    return x

In [ ]:
def generate_boundary_points():
    x_boundary = torch.tensor([1])


    return x_boundary.view(-1,1)


def generate_boundary_training_data():
    x_boundary = generate_boundary_points()


    return x_boundary

In [ ]:
def pde(x,model):

    x = x.double()

    # print('*********************************************************')
    # print(x.size())
    # print(x.dtype)

    input_data = x

    # print('*********************************************************')
    # print(input_data.size())
    # print(input_data.dtype)

    u = model(input_data)
    u_x = torch.autograd.grad(u, x ,grad_outputs= torch.ones_like(u), create_graph= True, retain_graph=True)[0]
    # u_xx = torch.autograd.grad(u_x,x,grad_outputs= torch.ones_like(u_x), create_graph= True, retain_graph=True)[0]
    # print('*********************************************************')
    # print(u_x.size())
    # print(u_x.dtype)
    # print('*********************************************************')
    # print(u.size())
    # print(u.dtype)
    # print('*********************************************************')
    # print(x.size())
    # print(x.dtype)
    eq_residual = 1 * u_x - 1 * u*x - u*u
    return eq_residual

In [ ]:
def train_PINN(model, num_iterations, num_points):
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    # breakpoint()
    for  iteration in range(num_iterations):
        optimizer.zero_grad()

        # breakpoint()

        x = generate_training_data(num_points)

        # breakpoint()

        x_b = generate_boundary_training_data()

        # breakpoint()

        custom_value = 0
        u_boundary_x = boundary_condition()

        # breakpoint()

        residual = pde(x, model)

        # breakpoint()

        u_boundary_x = u_boundary_x.double()
        x_b = x_b.double()

        # print('*********************************************************')
        # print(u_boundary_x.size())
        # print(u_boundary_x.dtype)
        # print('*********************************************************')
        # print(x_b.size())
        # print(x_b.dtype)
        # print('*********************************************************')

        # breakpoint()

        loss =  nn.MSELoss()(u_boundary_x, model(x_b)) + \
                nn.MSELoss()(residual, torch.zeros_like(residual))

        # breakpoint()

        loss.backward()
        optimizer.step()

        if iteration % 100 == 0:
            print("itration", iteration, "loss", loss )

In [ ]:
model = PINN()
num_iterations = 10000
num_points = 100
train_PINN(model, num_iterations, num_points)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


itration 0 loss tensor(1.0281, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 100 loss tensor(0.1173, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 200 loss tensor(0.0115, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 300 loss tensor(0.0013, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 400 loss tensor(0.0008, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 500 loss tensor(0.0004, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 600 loss tensor(0.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 700 loss tensor(9.9970e-05, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 800 loss tensor(8.1752e-05, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 900 loss tensor(6.4284e-05, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 1000 loss tensor(4.6437e-05, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 1100 loss tensor(6.1586e-05, dtype=torch.float64, grad_fn=<AddBackward0>)
itration 1200 loss tensor(5.2063e-05, dtype=

In [ ]:
t = torch.linspace(0,1,10).flatten()
X = torch.stack([t], dim=1)
X

tensor([[0.0000],
        [0.1111],
        [0.2222],
        [0.3333],
        [0.4444],
        [0.5556],
        [0.6667],
        [0.7778],
        [0.8889],
        [1.0000]])

In [ ]:
with torch.no_grad():
    x_vals = torch.linspace(0, 1, 50)


    input_data = torch.stack([x_vals.double()], dim=1)
    solution = model(input_data)
    sol = torch.stack([x_vals, solution.flatten()], dim=1)

    print(sol)



tensor([[0.0000, 0.3523],
        [0.0204, 0.3550],
        [0.0408, 0.3578],
        [0.0612, 0.3608],
        [0.0816, 0.3640],
        [0.1020, 0.3674],
        [0.1224, 0.3710],
        [0.1429, 0.3748],
        [0.1633, 0.3788],
        [0.1837, 0.3831],
        [0.2041, 0.3876],
        [0.2245, 0.3924],
        [0.2449, 0.3974],
        [0.2653, 0.4027],
        [0.2857, 0.4083],
        [0.3061, 0.4143],
        [0.3265, 0.4205],
        [0.3469, 0.4271],
        [0.3673, 0.4340],
        [0.3878, 0.4412],
        [0.4082, 0.4489],
        [0.4286, 0.4569],
        [0.4490, 0.4654],
        [0.4694, 0.4743],
        [0.4898, 0.4836],
        [0.5102, 0.4935],
        [0.5306, 0.5038],
        [0.5510, 0.5147],
        [0.5714, 0.5261],
        [0.5918, 0.5382],
        [0.6122, 0.5509],
        [0.6327, 0.5643],
        [0.6531, 0.5785],
        [0.6735, 0.5934],
        [0.6939, 0.6091],
        [0.7143, 0.6258],
        [0.7347, 0.6434],
        [0.7551, 0.6620],
        [0.7

In [ ]:
from sympy import symbols, Function, Derivative, dsolve, simplify, Eq, print_latex
from scipy.special import erfi
from math import exp, sqrt
import numpy as np

x   = symbols('x')
y   = Function('y')
ode = Eq(Derivative(y(x), x), y(x)*(x+y(x)))
soll = dsolve(ode, y(x), ics={y(1):1})
soll = simplify(soll.evalf())
print(soll)

Eq(y(x), -2.0*exp(x**2/2)/(2.506628274631*erfi(sqrt(2)*x/2) - 5.68735786522071))


In [ ]:
def ode_sol(x):
    return -2.0*exp(x**2/2)/(2.506628274631*erfi(sqrt(2)*x/2) - 5.68735786522071)

In [ ]:
sol_np = sol.numpy()
np.shape(sol_np)
points = sol_np[:, 0]
points

array([0.        , 0.02040816, 0.04081633, 0.06122449, 0.08163265,
       0.10204081, 0.12244898, 0.14285713, 0.1632653 , 0.18367347,
       0.20408162, 0.22448979, 0.24489796, 0.26530612, 0.28571427,
       0.30612245, 0.32653061, 0.34693876, 0.36734694, 0.3877551 ,
       0.40816325, 0.42857143, 0.44897959, 0.46938774, 0.48979592,
       0.51020408, 0.53061223, 0.55102044, 0.5714286 , 0.59183675,
       0.6122449 , 0.63265306, 0.65306121, 0.67346942, 0.69387758,
       0.71428573, 0.73469388, 0.75510204, 0.77551019, 0.79591835,
       0.81632656, 0.83673471, 0.85714287, 0.87755102, 0.89795917,
       0.91836733, 0.93877554, 0.95918369, 0.97959185, 1.        ])

In [ ]:
pred_y = []
true_y = []
pred_y = sol_np[:, 1]
for x in points:
    true_y.append(ode_sol(x))


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot

def plot_feature(true_y, pred_y, points):

    fig_prices = go.Figure()
    fig_prices.add_trace(go.Scatter(x=points, y=true_y,
                        # fill= 'tonexty',
                        mode='lines',
                        # line_color='indigo',
                        name="Точний розв'язок"))
    fig_prices.add_trace(go.Scatter(
                        x=points,
                        y=pred_y,
                        # fill='tonexty', # fill area between trace0 and trace1
                        mode='lines',
                        # line_color='red',
                        name="Наближений розв'язок"))


    fig_prices.update_layout(yaxis_title="Значення функції")
    fig_prices.update_layout(
    xaxis=dict(
        title='Точки колокації',
        title_font=dict(size=20),  # Збільшення розміру шрифту підпису осі X
        tickfont=dict(size=16),    # Збільшення розміру шрифту тікерів
    ),
    yaxis=dict(
        title='Значення функції',
        title_font=dict(size=20),  # Збільшення розміру шрифту підпису осі Y
        tickfont=dict(size=16),    # Збільшення розміру шрифту тікерів
    ),
    legend=dict(
        font=dict(size=18)          # Збільшення розміру шрифту легенди
    )
)
    fig_prices.show()

    return 0

In [ ]:
plot_feature(true_y, pred_y, points)

0

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(true_y, pred_y)

0.0010609372446527376